In [ ]:

library(ggplot2)
library(forecast)
library(dplyr)
library(colortools)

monthly_milk <- read.csv("C:\\Users\\pavan\\Downloads\\monthly_milk.csv")  # Milk production per cow per month
daily_milk <- read.csv("C:\\Users\\pavan\\Downloads\\daily_milk.csv")  # Milk production per cow per milking

head(monthly_milk)

class(monthly_milk)

class(monthly_milk$month)

head(daily_milk)

# Coerce to `Date` class
monthly_milk$month_date <- as.Date(monthly_milk$month, format = "%Y-%m-%d")

format(monthly_milk$month_date, format = "%Y-%B-%u")
class(format(monthly_milk$month_date, format = "%Y-%B-%u"))  # class is no longer `Date`





ggplot(monthly_milk, aes(x = month_date, y = milk_prod_per_cow_kg)) +
  geom_line() +
  scale_x_date(date_labels = "%Y", date_breaks = "1 year") +
  theme_classic()

ggplot(monthly_milk, aes(x = month_date, y = milk_prod_per_cow_kg)) +
  geom_line() +
  geom_smooth(method = "loess", se = FALSE, span = 0.6) +
  theme_classic()

#Play around with date_labels, replacing "%Y" with some other date marks from the 
#table above (e.g. %m-%Y). 
#date_breaks can also be customised to change the axis label frequency. 
#Other options include month, week and day (e.g. "5 weeks")

#span sets the number of points used to plot each local regression in the curve:
#  the smaller the number, 
#the more points are used and the more closely the curve will fit the original data.

# Extract month and year and store in separate columns
monthly_milk$year <- format(monthly_milk$month_date, format = "%Y")
monthly_milk$month_num <- format(monthly_milk$month_date, format = "%m")

# Create a colour palette using the `colortools` package 
year_pal <- sequential(color = "darkred", percentage = 5, what = "value")

# Make the plot
ggplot(monthly_milk, aes(x = month_num, y = milk_prod_per_cow_kg, group = year)) +
  geom_line(aes(colour = year)) +
  theme_classic() + 
  scale_color_manual(values = year_pal)

# 
# Next, it looks like there are some peaks and troughs that occur regularly in each year.
# This is a “seasonal” pattern. We can investigate this pattern more by plotting each year as 
# it’s own line and comparing the different years:

# Transform to `ts` class
monthly_milk_ts <- ts(monthly_milk$milk_prod, start = 1962, end = 1975, freq = 12)  # Specify start and end year, measurement frequency (monthly = 12)

# Decompose using `stl()`
monthly_milk_stl <- stl(monthly_milk_ts, s.window = "period")

# Generate plots
plot(monthly_milk_stl)  # top=original data, second=estimated seasonal, third=estimated smooth trend, bottom=estimated irregular element i.e. unaccounted for variation
monthplot(monthly_milk_ts, choice = "seasonal")  # variation in milk production for each month
seasonplot(monthly_milk_ts)

# forecasting
# ETS stands for Error, Trend, Seasonality.
# N = None
# A = Additive
# M = Multiplicative
# Z = Automatically selected

monthly_milk_model <- window(x = monthly_milk_ts, start = c(1962), end = c(1970))
monthly_milk_test <- window(x = monthly_milk_ts, start = c(1970))



# Creating model objects of each type of ets model
milk_ets_auto <- ets(monthly_milk_model) # model = "MMM"


# Creating forecast objects from the model objects
milk_ets_fc <- forecast(milk_ets_auto, h = 120)  # `h = 60` means that the forecast will be 60 time periods long, in our case a time period is one month


# Convert forecasts to data frames
milk_ets_fc_df <- cbind("Month" = rownames(as.data.frame(milk_ets_fc)), as.data.frame(milk_ets_fc))  # Creating a data frame
names(milk_ets_fc_df) <- gsub(" ", "_", names(milk_ets_fc_df))  # Removing whitespace from column names
milk_ets_fc_df$Date <- as.Date(paste("01-", milk_ets_fc_df$Month, sep = ""), format = "%d-%b %Y")  # prepending day of month to date
milk_ets_fc_df$Model <- rep("ets")  # Adding column of model type



# Combining into one data frame
forecast_all <- rbind(milk_ets_fc_df)

# Plotting with ggplot
ggplot() +
  geom_line(data = monthly_milk, aes(x = month_date, y = milk_prod_per_cow_kg)) +  # Plotting original data
  geom_line(data = forecast_all, aes(x = Date, y = Point_Forecast, colour = Model)) +  # Plotting model forecasts
  theme_classic()


accuracy(milk_ets_fc, monthly_milk_test)


milk_ets_fc_df %>%
  filter(Month == "Jan 1980") %>%
  select(Month, Point_Forecast)






